<a href="https://colab.research.google.com/github/namuunbayar/MachineLearning/blob/main/20231113_kinliser_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Google drive mount



先にGoogle Driveで「共有アイテム」 -> 「kiliser」を選択して右クリック後「ドライブにショートカットに追加」を実行。
こうすることで、共有アイテムもマウントできるようになる。

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/kinliser

/content/drive/MyDrive/otas_baito/kinliser


In [ ]:
%ls

20231113_kinliser_test.ipynb            noritz/           osaka_gas/  tokyo_gas/
gas-meter-ocr-356007-981ec8805f98.json  not_categorized/  other/      yolov8_meter.ipynb
json_response/                          ocr_test.ipynb    rinnai/


In [ ]:
!pip install -q google-cloud-vision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.1/444.1 kB 5.3 MB/s eta 0:00:00


In [85]:
# responseをjsonで保存
import json
def save_as_json(response, filename):
    data = AnnotateImageResponse.to_json(response)
    with open(filename, mode='wt', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)

# jsonをAnnotateImageResponse形式で読み出し
def load_from_json(filename):
    with open(filename, mode='r', encoding='utf-8') as file:
        temp = json.load(file)
    response = AnnotateImageResponse.from_json(temp)
    return response

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ="./gas-meter-ocr-356007-981ec8805f98.json"

In [84]:
from google.cloud import vision
from google.cloud.vision import AnnotateImageResponse
import json
import io

#Vision APIを呼び出しjsonで保存
def get_resp(file_path, output_dir = 'json_response/', hint='ja'):
    # Instantiates a client
    client = vision.ImageAnnotatorClient()

    # The name of the image file to annotate
    file_name = os.path.abspath(file_path)

    # Loads the image into memory
    with io.open(file_name, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # Performs label detection on the image file
    response =  client.document_text_detection(
            image=image,
            image_context={'language_hints': [hint]}
        )

    resp_file = output_dir
    basename = os.path.splitext(os.path.basename(file_path))[0]
    resp_file = resp_file + basename + '.json'

    save_as_json(response, resp_file)

    return response

In [55]:
import pathlib

input_dir = 'not_categorized'
image_list = list(pathlib.Path(input_dir).glob('**/*.jpg'))

In [ ]:
image_list[0]

PosixPath('not_categorized/202309128590_1 (1).jpg')

In [ ]:
# 実行済みのため、変更・実行する必要なし
# for image_path in image_list:
#     get_resp(image_path)

In [56]:
json_dir = 'json_response'
json_list = list(pathlib.Path(json_dir).glob('**/*.json'))

In [59]:
json_list[0]


PosixPath('json_response/202309128590_1.json')

In [71]:
  response = load_from_json(json_list[0])
  print(response.text_annotations[0].description)

Ø
禁止
換気注意
(屋内式FEのみ)
使用中は換気に十分注意する。 但
レンジフード 換気扇を回さな
い。 一酸化炭素中毒の恐れあり。
波板囲い禁止
I 波板などで囲まない。
一酸化炭素中毒や火災の恐れあり。
屋外式ガス機器本体
または屋内式ガス機器
禁止 (給排気筒トップ
波板囲い
型式 GQ-1628WS-TB
設置の方式 屋外式
都市ガス用
定格電圧
定格周波数
定格消費電力
(88)
JIA 18 LE
13A
12A
33.4kW 31.1kW
AC100V
50/60Hz
40/40W
07.06 - 054358
株式会社ノーリツ
凍結予防ヒーター
125W
認証登録番号 OW1004
水道法基準適合
PS B
TG
スターリビング・
ZF12 03488
●遠隔操作の場合は取扱説明書に記載されているリモコン
をご使用ください。
●リモコンを接続した場合はリモコンの表示に従って操作
し着火・消火をリモコンの表示により確認してください。
●機器内に長時間たまっていた水は、飲用または調理に用
いないでください。
点検と手入
ときどき取扱説明書に記載してある要領で、
れを行ってください。
●冬期、冷え込みが厳しく凍結の恐れがあるときは、取扱
説明書に記載してある方法で凍結予防をしてください。
これを怠りますと、 凍結して機器が破損する場合があり
ます。
●機器を移設される場合は、移転先のガス事業者、 販売店
または取扱説明書記載の窓口までご連絡ください。
●異常時(排気口から煙が出る、 こげ臭など) や緊急時
(地震・火災など) は使用をやめ、 販売店または取扱説
明書記載の窓口までご連絡ください。
DNSK005 ②
~ [設置上の注意]
・屋内に絶対に設置しないでください。
区分名 D
エネルギー消費効率 82.5%
株式会社ノーリツ


In [89]:
import re

# テキストからGQ-1628WS-TBのような形式の文字列を抽出する正規表現パターン
pattern = r'\b[A-Z]+-?\d+[A-Z]*-?\w+\b'
j=0
for jsan in json_list:
  response = load_from_json(jsan)
  # テスト用の文字列
  text = response.text_annotations[0].description



# 正規表現にマッチする部分を抽出
  match = re.search(pattern, text)

# 抽出した文字列を表示
  if match:
      extracted_string = match.group()
      print(f"{jsan}: {extracted_string}")
      j += 1
  else:
      print(f"{jsan}:該当する文字列は見つかりませんでした。")
print(j/len(json_list))

json_response/202309128590_1.json: GQ-1628WS-TB
json_response/202309138595_1.json: FT4204ARSSW3Q
json_response/202309138594_1.json: GT-2450AWX
json_response/202309128593_1.json: FT4207ARS2AW3CU
json_response/202309128591_1.json: GS-2000W-1
json_response/202309128592_1.json: GFK-1616A
json_response/202309138603_1.json: A2000AW
json_response/202309138600_1.json: AD-2899AFQ
json_response/202309138596_1.json: N076A
json_response/202309138605_1.json: A2000SAW-E
json_response/202309138601_1.json: FH-20AWD
json_response/202309138602_1.json: AT-366RSA-AW2Q
json_response/202309138599_1.json: U217-941X01
json_response/202309138604_1.json: YG2432RT
json_response/202309138598_1.json: V2405SAT
json_response/202309148606_1.json: GRO-1628SAX-1
json_response/202309148610_1.json: E6771
json_response/202309148611_1.json: RUX-1600UPSOTL
json_response/202309148607_1.json: V2401AW
json_response/202309148608_1.json: EZMK005
json_response/202309178663_1.json: N436A
json_response/202309178664_1.json:該当する文字列は見

In [90]:
0.982725527831094*521


512.0